#Cleaning script for incoming DM form. Process to include:
# a) First clean and renaming of columns from raw form
# b) filter by watershed and ACO flight number
# c) Adding of coordinate information and importing of GNSS data 

In [34]:
#import libraries
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [36]:
#flight/phase number?
flt_no = '2'

In [37]:
#open the raw csv, select specific columns to keep, drop additional columns
path = r"G:\ACO\2024"
data = pd.read_csv(path + r"\1_scripts\working\p1\MV_24_P01_PM.csv", encoding='mac-roman')
df = pd.DataFrame(data)

In [38]:
data = df[df["Study_Area"].notna()]
len(df.index)

98

In [49]:
df

df.to_csv(path + "\\test_csv.csv", index=False)

In [40]:
#put columns to keep in to a list form
columns = df[["Survey_Start_Time",
          "username", 
          "Study_Area", 
          "Other_Study_Area", 
          "User_s_", 
          "Plot_ID", 
          "Tube_Name", 
          "Pre_Survey_Notes", 
          "GNSS_Used_", 
          "GNSS_Setup", 
          "Other_GNSS_Setup",
          "GNSS_Status",
          "GNSS_Height_Rover_to_Snow__cm_", 
          "Tare_Weight__g_",
          "Tare_Weight__cm_", 
          "Type_of_Plot", 
          "Point_Observation.Cardinal_Direction", 
          "Point_Observation.Distance_From_Centre__m_",
          "Point_Observation.Custom_Distance_From_Centre__m_",
          "Point_Observation.Plot_Features", 
          "Point_Observation.Sample_Type", 
          "Point_Observation.Depth__cm_", 
          "Point_Observation.Depth__cm_:timestamp", 
          "Point_Observation.Depth_Final__cm_",
          "Point_Observation.Core_Length__cm_", 
          "Point_Observation.Plug__cm_", 
          "Point_Observation.SWE_cm",
          "Point_Observation.Mass___Tube__g_",
          "Point_Observation.Multi_Part_Core_",
          "Point_Observation.Multi_Part_Core_Section_Number", 
          "Point_Observation.Additional_Measurements.Multi_Part_Core_Section_Number_copy",
          "Point_Observation.Additional_Measurements.New_Depth__cm_", 
          "Point_Observation.Additional_Measurements.New_Depth__cm_:timestamp", 
          "Point_Observation.Additional_Measurements.Core_Section_Length__cm_",
          "Point_Observation.Additional_Measurements.Plug_", 
          "Point_Observation.Additional_Measurements.SWE",
          "Point_Observation.Additional_Measurements.Mass___Tube", 
          "Point_Observation.Core_Length_Final__cm_",
          "Point_Observation.Core_Features", 
          "Point_Observation.Depth_of_Saturation", 
          "Point_Observation.Mass_Final__g_", 
          "Point_Observation.SWE_Final__cm_",    
          "Point_Observation.Depth_Max",
          "Point_Observation.Retrieval____", 
          "Point_Observation.SWE__cm_", 
          "Point_Observation.Density", 
          "Point_Observation.Density_MetroVan",
          "Point_Observation.Sample_Rating", 
          "Point_Observation.Point_Observation_Notes", 
          "Snow_Pit_Measurement.Distance_from_centre_of_plot__m_", 
          "Snow_Pit_Measurement.Depth_above_ground__cm_", 
          "Snow_Pit_Measurement.Temperature___C_",
          "Snow_Pit_Measurement.Density_Notes", 
          "Survey_End_TIme"]].copy()

In [41]:
#change the column names to manageable names, ie. 
cols = df.rename(columns={
                    "Survey_Start_Time": "plot_datetime",
                    "username": "user_name",
                    "Study_Area": "study_area",
                    "Other_Study_Area": "other_study_area",
                    "User_s_": "users",
                    "Plot_ID": "plot_id",
                    "Tube_Name": "tube_name",
                    "Pre_Survey_Notes": "pre_survey_notes",
                    "GNSS_Used_": "gnss_unit",
                    "GNSS_Setup": "gnss_setup",
                    "Other_GNSS_Setup": "other_gnss_setup",
                    "GNSS_Status": "gnss_status",
                    "GNSS_Height_Rover_to_Snow__cm_": "rover_height",
                    "Tare_Weight__g_": "tare_weight_g",
                    "Tare_Weight__cm_": "tare_weight_cm",
                    "Type_of_Plot": "plot_type",
                    "Point_Observation.Cardinal_Direction": "cardinal",
                    "Point_Observation.Distance_From_Centre__m_": "distance_m",
                    "Point_Observation.Custom_Distance_From_Centre__m_": "custom_distance",
                    "Point_Observation.Plot_Features": "plot_features",
                    "Point_Observation.Sample_Type": "sample_type",
                    "Point_Observation.Depth__cm_": "depth_cm",
                    "Point_Observation.Depth__cm_:timestamp": "depth_timestamp",
                    "Point_Observation.Depth_Final__cm_": "depth_final_cm",
                    "Point_Observation.Core_Length__cm_": "core_length_cm",
                    "Point_Observation.Plug__cm_": "plug_cm",
                    "Point_Observation.SWE_cm": "swe_cm",
                    "Point_Observation.Mass___Tube__g_": "mass_tube_g_drop",
                    "Point_Observation.Multi_Part_Core_": "multi_core",
                    "Point_Observation.Multi_Part_Core_Section_Number": "multi_part_core_num",
                    "Point_Observation.Additional_Measurements.Multi_Part_Core_Section_Number_copy": "multi_core_num_copy",
                    "Point_Observation.Additional_Measurements.New_Depth__cm_": "depth_cm_new",
                    "Point_Observation.Additional_Measurements.New_Depth__cm_:timestamp": "depth_cm_timestamp_new",
                    "Point_Observation.Additional_Measurements.Core_Section_Length__cm_": "core_section_length_cm",
                    "Point_Observation.Additional_Measurements.Plug_": "plug_cm_copy",
                    "Point_Observation.Additional_Measurements.Mass___Tube": "mass_tube_g_new",
                    "Point_Observation.Core_Length_Final__cm_": "core_length_final",
                    "Point_Observation.Core_Features": "core_features",
                    "Point_Observation.Depth_of_Saturation": "depth_of_saturation",
                    "Point_Observation.Mass_Final__g_": "mass_final_g",
                    "Point_Observation.SWE_Final__cm_": "swe_final_cm",
                    "Point_Observation.Depth_Max": "depth_max",
                    "Point_Observation.Retrieval____": "retrieval",
                    "Point_Observation.SWE__cm_": "swe_cm_drop",
                    "Point_Observation.Density": "density_drop",
                    "Point_Observation.Density_MetroVan": "density",
                    "Point_Observation.Sample_Rating": "sample_rating",
                    "Point_Observation.Point_Observation_Notes": "notes",
                    "Snow_Pit_Measurement.Distance_from_centre_of_plot__m_": "snow_pit_distance_from_centre",
                    "Snow_Pit_Measurement.Depth_above_ground__cm_": "snow_pit_depth_above_ground",
                    "Snow_Pit_Measurement.Temperature___C_": "snow_pit_temperature",
                    "Snow_Pit_Measurement.Density_Notes": "snow_pit_density",
                    "Survey_End_TIme": "survey_end"
                    }, inplace=True)

In [50]:
new_df = df.assign(aco_flight_number= str(flt_no))

In [51]:
aco_pos = new_df.pop('aco_flight_number')
new_df.insert(0, 'aco_flight_number', aco_pos)

In [55]:
new_df

df.to_csv(path + "\\test_3.csv", index=False)

In [53]:
df = new_df.drop(columns=['submissionid', 
                                'submissiondatetime', 
                                'deviceid', 
                                'user_name', 
                                'Sampling_Design',
                                'Point_Observation.Abbreviations', 
                                'Point_Observation.Picture', 
                                'Point_Observation.Number_of_probe_extensions_used_',
                                'mass_tube_g_drop',
                                'mass_final_g',
                                'swe_cm_drop',
                                'density_drop'])

#Take the cleaned form and filter out by watershed
#export summary statistics

In [56]:
#join depth_final_cm and depth_max - name snow_depth
columns = df.loc[:, ['depth_final_cm', 'depth_max']]
columns.fillna(0, inplace=True)
df= df.assign(snow_depth = (columns['depth_final_cm']) + (columns['depth_max']))

In [22]:
#titles = list(df.columns)
#titles

In [57]:
df.insert(25, "snow_depth", df.pop("snow_depth"))

In [58]:
len(df.index)

98

In [59]:
#export the clean raw device magic form to a csv
df.to_csv("processing_data\DM_trip" + str(flt_no) + "_clean.csv", index = False)

In [37]:
#filter by watershed: "Russell Creek", "Cruickshank",or "Englishman"
cru_df =df[df['study_area'] == 'Cruickshank']
eng_df = df[df['study_area'] == 'Englishman']
rus_df = df[df['study_area'] == 'Russell Creek']
mv_df = df[df['study_area'] == 'Metro Vancouver']

In [38]:
mv_df

,aco_flight_number,plot_datetime,study_area,other_study_area,users,plot_id,tube_name,pre_survey_notes,gnss_unit,gnss_setup,...,retrieval,swe_cm,density,sample_rating,notes,snow_pit_distance_from_centre,snow_pit_depth_above_ground,snow_pit_temperature,snow_pit_density,survey_end
0,1,5/2/2024 12:37,Metro Vancouver,NaN,Ak bf tm,4EO,CHRL 1,40-60 cm footprints,No will survey later,NaN,...,NaN,NaN,NaN,Good,NaN,NaN,NaN,NaN,NaN,5/2/2024 13:09
1,1,5/2/2024 12:37,Metro Vancouver,NaN,Ak bf tm,4EO,CHRL 1,40-60 cm footprints,No will survey later,NaN,...,NaN,NaN,NaN,Good,NaN,NaN,NaN,NaN,NaN,5/2/2024 13:09
2,1,5/2/2024 12:37,Metro Vancouver,NaN,Ak bf tm,4EO,CHRL 1,40-60 cm footprints,No will survey later,NaN,...,NaN,NaN,NaN,Good,NaN,NaN,NaN,NaN,NaN,5/2/2024 13:09
3,1,5/2/2024 12:37,Metro Vancouver,NaN,Ak bf tm,4EO,CHRL 1,40-60 cm footprints,No will survey later,NaN,...,NaN,NaN,NaN,Good,NaN,NaN,NaN,NaN,NaN,5/2/2024 13:09
4,1,5/2/2024 12:37,Metro Vancouver,NaN,Ak bf tm,4EO,CHRL 1,40-60 cm footprints,No will survey later,NaN,...,NaN,NaN,NaN,Good,NaN,NaN,NaN,NaN,NaN,5/2/2024 13:09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,1,5/2/2024 13:53,Metro Vancouver,NaN,Ak bf tm,5NO,CHRL 1,Other team took notes for other measurements. ...,No will survey later,NaN,...,NaN,NaN,NaN,Good,NaN,NaN,NaN,NaN,NaN,5/2/2024 14:05
241,1,5/2/2024 13:53,Metro Vancouver,NaN,Ak bf tm,5NO,CHRL 1,Other team took notes for other measurements. ...,No will survey later,NaN,...,NaN,NaN,NaN,Good,NaN,NaN,NaN,NaN,NaN,5/2/2024 14:05
242,1,5/2/2024 13:53,Metro Vancouver,NaN,Ak bf tm,5NO,CHRL 1,Other team took notes for other measurements. ...,No will survey later,NaN,...,NaN,NaN,NaN,Good,NaN,NaN,NaN,NaN,NaN,5/2/2024 14:05
243,1,5/2/2024 13:53,Metro Vancouver,NaN,Ak bf tm,5NO,CHRL 1,Other team took notes for other measurements. ...,No will survey later,NaN,...,NaN,NaN,NaN,Good,NaN,NaN,NaN,NaN,NaN,5/2/2024 14:05


In [56]:
#group by "plot_id", export csv of summary stats for snow. 
#Stats that are nice to include are: 
#snow depth:count, average, st dev; density: count, avg, st dev, & SWE : count, avg, st.dev



###CRUICKSHANK###

cru_sum = cru_df.groupby(['aco_flight_number',
    'plot_id']).agg({
    "snow_depth": ["mean", "median", "std", "count"],
    "density": ["mean", "median", "std", "count"],
    "swe_cm": ["mean", "median", "std", "count"]
})

#check if directory exists
if not os.path.isdir(path + r"\Cruickshank\4_field_data\plots\working\P" + str(flt_no)):
    #if the folder path is not presnet, then create it
    os.makedirs(path + r"\Cruickshank\4_field_data\plots\working\P" + str(flt_no))
    
#Export the filtered summary stats to created folder
cru_sum.to_csv(path + "\Cruickshank\\4_field_data\plots\working\P" + str(flt_no) + "\DM_trip" + str(flt_no) +"_cru_sum.csv", index = True)

In [23]:
###ENGLISHMAN###

eng_sum = eng_df.groupby(['aco_flight_number',
    'plot_id']).agg({
    "snow_depth": ["mean", "median", "std", "count"],
    "density": ["mean", "median", "std", "count"],
    "swe_cm": ["mean", "median", "std", "count"]
})

#check if directory exists
if not os.path.isdir(path + r"\Englishman\4_field_data\plots\working\P" + str(flt_no)):
    #if the folder path is not presnet, then create it
    os.makedirs(path + r"\Englishman\4_field_data\plots\working\P" + str(flt_no))
    
#Export the filtered summary stats to created folder
eng_sum.to_csv(path + "\Englishman\\4_field_data\plots\working\P" + str(flt_no) + "\DM_trip" + str(flt_no) +"_eng_sum.csv", index = True)

In [17]:
###TSITIKA####


rus_sum = rus_df.groupby(['aco_flight_number',
    'plot_id']).agg({
    "snow_depth": ["mean", "median", "std", "count"],
    "density": ["mean", "median", "std", "count"],
    "swe_cm": ["mean", "median", "std", "count"]
})

#check if directory exists
if not os.path.isdir(path + r"\Tsitika\4_field_data\plots\working\P" + str(flt_no)):
    #if the folder path is not presnet, then create it
    os.makedirs(path + r"\Tsitika\4_field_data\plots\working\P" + str(flt_no))
    
#Export the filtered summary stats to created folder
rus_sum.to_csv(path + "\Tsitika\\4_field_data\plots\working\P" + str(flt_no) + "\DM_trip" + str(flt_no) +"_rus_sum.csv", index = True)